# テストの実行

In [131]:
import os

import time
import datetime

import subprocess
import logging
from concurrent import futures

import numpy as np
import pandas as pd

In [132]:
TESTSET_DIR = os.path.join('/home', 'jupyter', 'work', '01_testset')
PRJ_DIR = os.path.join('/home', 'jupyter', 'work')

## マスタの読み込み

In [133]:
pre_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '01_test_pre.csv'), usecols=['seed'])
sys_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '02_test_sys.csv'), usecols=['seed'])
stress_seed_df = pd.read_csv(os.path.join(TESTSET_DIR, '03_test_stress.csv'), usecols=['seed'])

## 実行するロジックの指定

In [136]:
# 実行プログラムにタグをつけておく
PROG_TAG = 'min_tree_6s'

In [137]:
def solve(seed, N, M, D, K, problem, prog_path):
    start_time = time.perf_counter()
    
    command_str = 'echo {} | {}'.format(problem, prog_path)
    res = subprocess.run(command_str, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    # 経過時間(ミリ秒単位)
    e_time = time.perf_counter() - start_time
    e_time = int(1000 * e_time)    
    #print('{}'.format(prob_id))    
    return (seed, N, M, D, K, e_time, res)

In [142]:
def run_test(testset_name):
    result_df = pd.DataFrame()
    future_list = []

    logger.info('Start')

    testset_path = os.path.join(TESTSET_DIR, testset_name+'.csv')
    testset_df = pd.read_csv(testset_path)

    with futures.ThreadPoolExecutor(max_workers=9) as executor:
        for _, row in testset_df.iterrows():
            seed, N, M, D, K, problem = row

            # バッチ実行
            future = executor.submit(solve, seed=seed,  N=N, M=M, D=D, K=K, problem=problem, prog_path=prog_path)
            future_list.append(future)

        _ = futures.as_completed(fs=future_list)

    for future in future_list:
        seed, N, M, D, K, e_time, res = future.result()

        # 結果をまとめる
        solve_result = []
        
        solve_result.append(testset_name)

        # 問題パラメタ
        solve_result.append(seed)
        solve_result.append(N)
        solve_result.append(M)
        solve_result.append(D)
        solve_result.append(K)

        # 経過時間
        solve_result.append(e_time)
        
        try:
            # コスト
            cost = int(res.stderr.decode('utf-8').split()[-5].replace('Cost=', ''))

            # 非連結なノードペア数
            discon_cnt = int(res.stderr.decode('utf-8').split()[-4].replace('DisconCnt=', ''))

            # 工事辺数のオーバー
            over_k_cnt = int(res.stderr.decode('utf-8').split()[-3].replace('OverK=', ''))
            
            # 迂回路中の辺の数
            in_bypass_cnt = int(res.stderr.decode('utf-8').split()[-2].replace('InBypass=', ''))

            iter_cnt = int(res.stderr.decode('utf-8').split()[-1].replace('Iter=', ''))
                       
        except Exception as e:
            print('Error: seed={}'.format(seed))
            print(e)
            return

        solve_result.append(cost)
        solve_result.append(discon_cnt)
        solve_result.append(over_k_cnt)
        solve_result.append(in_bypass_cnt)
        solve_result.append(iter_cnt)

        result_df = pd.concat([result_df, pd.DataFrame(solve_result).T], axis=0)

    logger.info('finish!')
    
    # 結果を整形
    result_df.index = range(result_df.shape[0])
    result_df.columns = ['testset', 'seed', 'N', 'M', 'D', 'K', 'time', 'cost', 'discon_cnt', 'over_k_cnt', 'in_bypass_cnt', 'iter_cnt']

    return result_df

In [143]:
logger = logging.getLogger(__name__)

fmt = "%(asctime)s: %(message)s"
logging.basicConfig(level=logging.INFO, format=fmt)

In [177]:
def get_summary_df(result_df):
    # 全体サマリ
    summary_all_df = pd.DataFrame()

    for testset in np.unique(result_df['testset']):
        test_result_df = result_df.query('testset == "{}"'.format(testset))

        summary_df = pd.DataFrame(
        {
            'testset': [testset],
            
            'time_mean': [int(np.mean(test_result_df['time']))],
            
            'cost_mean': [int(np.mean(test_result_df['cost']))],
            'cost_min': [min(test_result_df['cost'])],
            'cost_max': [max(test_result_df['cost'])],

            'discon_cnt_mean': [np.mean(test_result_df['discon_cnt'])],
            'discon_cnt_max': [max(test_result_df['discon_cnt'])],

            'over_k_cnt_max': [max(test_result_df['over_k_cnt'])],

            'in_bypass_cnt_mean': [np.mean(test_result_df['in_bypass_cnt'])],
            'in_bypass_cnt_max': [max(test_result_df['in_bypass_cnt'])],

            'iter_cnt_mean': [np.mean(test_result_df['iter_cnt'])],

            'time_max': [max(test_result_df['time'])],
        })

        summary_all_df = pd.concat([summary_all_df, summary_df], axis=0)   

    summary_all_df['tag'] = PROG_TAG
    summary_all_df = summary_all_df[['tag', 'testset', 'time_mean', 
                                     'cost_mean', 'cost_min', 'cost_max',
                                     'discon_cnt_mean', 'discon_cnt_max',
                                     'over_k_cnt_max',
                                     'in_bypass_cnt_mean', 'in_bypass_cnt_max',
                                     'iter_cnt_mean',
                                     'time_max']]
    
    return summary_all_df

In [196]:
PROG_NAME_LIST = ['main_test']
#PROG_NAME_LIST = ['main', 'main_off']

#testset_name = '00_test_mini'
#testset_name = '01_test_pre'
testset_name = '02_test_sys'
#testset_name = '03_test_stress'

result_dict = {}
summary_all_dict = {}

for PROG_NAME in PROG_NAME_LIST:
    prog_path = os.path.join(PRJ_DIR, PROG_NAME)
    
    result_df = pd.DataFrame()
    
    testset_result_df = run_test(testset_name)
    result_df = pd.concat([result_df, testset_result_df], axis=0)
    
    result_dict[PROG_NAME] = result_df
    summary_all_dict[PROG_NAME] = get_summary_df(result_df)    

2023-02-11 08:15:11,084: Start
2023-02-11 08:40:14,466: finish!


In [197]:
# pre test
result_sub_df = pd.merge(pre_seed_df, result_df, on='seed')
summary_df = get_summary_df(result_sub_df)
summary_df['testset'] = '01_test_pre'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,over_k_cnt_max,in_bypass_cnt_mean,in_bypass_cnt_max,iter_cnt_mean,time_max
0,min_tree_6s,01_test_pre,6993,8252258,1814996,25517687,0.0,0,0,-1.0,-1,86506.2,9142


In [198]:
# sys test
result_sub_df = pd.merge(sys_seed_df, result_df, on='seed')
summary_df = get_summary_df(result_sub_df)
summary_df['testset'] = '02_test_sys'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,over_k_cnt_max,in_bypass_cnt_mean,in_bypass_cnt_max,iter_cnt_mean,time_max
0,min_tree_6s,02_test_sys,6745,10381085,1621008,60381799,0.0,0,0,-1.0,-1,93003.532,9948


In [188]:
# pre test
result_sub_df = pd.merge(pre_seed_df, result_df, on='seed')
summary_df = get_summary_df(result_sub_df)
summary_df['testset'] = '01_test_pre'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,over_k_cnt_max,in_bypass_cnt_mean,in_bypass_cnt_max,iter_cnt_mean,time_max
0,min_tree_6s,01_test_pre,6968,8316081,1864332,25647641,0.0,0,0,-1.0,-1,53953.12,9193


In [185]:
# pre test
result_sub_df = pd.merge(pre_seed_df, result_df, on='seed')
summary_df = get_summary_df(result_sub_df)
summary_df['testset'] = '01_test_pre'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,over_k_cnt_max,in_bypass_cnt_mean,in_bypass_cnt_max,iter_cnt_mean,time_max
0,min_tree_6s,01_test_pre,6942,8306971,1870322,25581121,0.0,0,0,-1.0,-1,53581.78,8949


In [182]:
# sys test
result_sub_df = pd.merge(sys_seed_df, result_df, on='seed')
summary_df = get_summary_df(result_sub_df)
summary_df['testset'] = '02_test_sys'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,over_k_cnt_max,in_bypass_cnt_mean,in_bypass_cnt_max,iter_cnt_mean,time_max
0,min_tree_6s,02_test_sys,6607,10806653,1687526,67757888,0.0,0,0,-1.0,-1,54947.196,9807


In [168]:
result_df.query('discon_cnt > 0')

,testset,seed,N,M,D,K,time,cost,discon_cnt,over_k_cnt,in_bypass_cnt,iter_cnt
13,02_test_sys,2122,847,1659,13,162,6690,204033747,1692,1,-1,10579
25,02_test_sys,3279,934,1786,29,66,8085,84691107,1866,1,-1,61765
26,02_test_sys,6407,526,925,9,128,5938,2994645155,7320,1,-1,167747
46,02_test_sys,5340,750,1326,9,165,6103,2115094602,10456,1,-1,134943
65,02_test_sys,2524,627,1096,13,107,6109,281330264,1252,1,-1,438099
...,...,...,...,...,...,...,...,...,...,...,...,...
1954,02_test_sys,8324,887,1868,10,195,6401,249262041,1772,1,-1,240207
1955,02_test_sys,8798,966,1741,6,322,6326,2147797142,11558,1,-1,162840
1983,02_test_sys,8451,841,1440,12,122,6432,633764106,5036,1,-1,6750
1988,02_test_sys,7786,920,1554,8,233,6315,607423554,3674,1,-1,11842


In [16]:
# sys test
summary_df = pd.DataFrame()

if testset_name == '02_test_sys' or testset_name == '03_test_stress':
    result_sub_df = pd.merge(sys_seed_df, result_df, on='seed')
    summary_df = get_summary_df(result_sub_df)
    summary_df['testset'] = '02_test_sys'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,over_k_cnt_max,in_bypass_cnt_mean,in_bypass_cnt_max,time_max
0,face_group_6s,02_test_sys,2606,17645946,3220054,107948938,0.0,0,0,-1.0,-1,5635


In [25]:
result_df.query('discon_cnt>0')

,testset,seed,N,M,D,K,time,cost,discon_cnt,over_k_cnt,in_bypass_cnt


In [15]:
# sys test
summary_df = pd.DataFrame()

if testset_name == '02_test_sys' or testset_name == '03_test_stress':
    result_sub_df = pd.merge(sys_seed_df, result_df, on='seed')
    summary_df = get_summary_df(result_sub_df)
    summary_df['testset'] = '02_test_sys'

summary_df

,tag,testset,time_mean,cost_mean,cost_min,cost_max,discon_cnt_mean,discon_cnt_max,over_k_cnt_max,in_bypass_cnt_mean,in_bypass_cnt_max,time_max
0,face_group_min_max_bet_1k,02_test_sys,1900,32585309,7264339,190856266,0.0,0,5,44.946,122,5297


In [100]:
# stress test
summary_df = pd.DataFrame()

if testset_name == '03_test_stress':
    result_sub_df = pd.merge(stress_seed_df, result_df, on='seed')
    summary_df = get_summary_df(result_sub_df)
    summary_df['testset'] = '03_test_stress'

summary_df

""


In [101]:
result_df.sort_values('N')

,testset,seed,N,M,D,K,time,cost,discon_cnt,over_k_cnt,in_bypass_cnt
1502,02_test_sys,7427,500,957,20,52,642,13808992,0,0,0
1207,02_test_sys,4739,500,1334,8,202,475,17163812,0,0,0
1816,02_test_sys,292,501,961,23,59,648,14828943,0,0,0
264,02_test_sys,5148,501,921,16,65,505,22920121,0,0,0
1834,02_test_sys,2718,501,913,19,51,630,19024381,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1599,02_test_sys,6254,999,2133,5,602,5290,45190397,0,0,9
103,02_test_sys,6401,1000,1992,22,95,2602,11331751,0,0,0
682,02_test_sys,1419,1000,1692,16,206,1672,29209255,0,0,0
1929,02_test_sys,6899,1000,1713,13,261,1282,32970926,0,0,0


# 結果ログの保存

In [15]:
result_df

,testset,seed,N,M,D,K,time,cost,discon_cnt
0,03_test_stress,0,791,2277,11,361,2651,10447535,0
1,03_test_stress,1,623,1146,25,70,784,14323787,0
2,03_test_stress,2,898,1576,8,356,1297,614052174,3584
3,03_test_stress,3,830,1605,12,209,1001,26342149,0
4,03_test_stress,4,700,1500,14,112,823,16142516,0
...,...,...,...,...,...,...,...,...,...
9995,03_test_stress,9995,879,1701,7,384,1241,50195819,0
9996,03_test_stress,9996,878,1532,15,115,1172,330432751,3504
9997,03_test_stress,9997,731,1537,30,75,1424,98632378,1460
9998,03_test_stress,9998,742,1778,5,614,1204,45966030,0


In [29]:
f = (result_df['M'] - result_df['M'] / result_df['D']) < result_df['N']

In [30]:
sum(f)

0

In [102]:
t_now = datetime.datetime.now() + datetime.timedelta(hours=9)
time_str = t_now.strftime('%Y%m%d_%H%M')

for PROG_NAME in PROG_NAME_LIST:
    result_df = result_dict[PROG_NAME]
    
    for testset in np.unique(result_df['testset']):
        csv_df = result_df.query('testset == "{}"'.format(testset))
        csv_df.to_csv(PRJ_DIR+'/result/{}_{}_{}_{}.csv'.format(time_str,PROG_TAG, testset, PROG_NAME), index=False)